In [ ]:
!pip install japanize-matplotlib

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

def load_and_preprocess_data(csv_file):
    df = pd.read_csv(csv_file, encoding='shift_jis')
    dependent_variable_col = df.columns[0]  
    condition_col = df.columns[1]  
    participant_col = df.columns[2]  
    session_col = df.columns[3]  

    df['condition_change'] = (df[condition_col] != df[condition_col].shift()).cumsum()
    df['independent_session'] = df[condition_col] + '_' + df['condition_change'].astype(str)
    
    return df, dependent_variable_col, condition_col, participant_col, session_col

def plot_abc_bc_patient_data(csv_file):
    sns.set(
        font='IPAexGothic',
        palette='Set2',
        context='paper'
    )
    
    df, dependent_variable_col, condition_col, participant_col, session_col = load_and_preprocess_data(csv_file)
    
    color_map = {condition: sns.color_palette("Set2")[i] for i, condition in enumerate(df[condition_col].unique())}

    for participant in df[participant_col].unique():
        participant_data = df[df[participant_col] == participant]
        plt.figure(figsize=(10, 4))

        for session in participant_data['independent_session'].unique():
            session_data = participant_data[participant_data['independent_session'] == session]
            condition = session.split('_')[0]  
            sns.lineplot(data=session_data, x=session_col, y=dependent_variable_col, marker='o', 
                         linewidth=2.5, markersize=8, color=color_map[condition], label=condition)

        for i in range(1, len(participant_data)):
            if participant_data['independent_session'].iloc[i] != participant_data['independent_session'].iloc[i - 1]:
                plt.axvline(x=participant_data[session_col].iloc[i] - 0.5, color='red', linestyle='--', linewidth=2)

        plt.xlabel(session_col, fontsize=16)
        plt.ylabel(dependent_variable_col, fontsize=16)
        plt.tick_params(axis='both', labelsize=14)
        
        plt.grid(color='grey', linestyle='--', linewidth=0.7, alpha=0.7)

        plt.xticks(rotation=45)  
        plt.yticks(fontsize=12)
        
        plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))  
        plt.gca().yaxis.set_major_locator(plt.MaxNLocator(nbins=8))  

        handles, labels = plt.gca().get_legend_handles_labels()
        unique_labels = list(df[condition_col].unique())
        by_label = dict(zip(labels, handles))
        plt.legend([by_label[label] for label in by_label if label.split('_')[0] in unique_labels],
                   unique_labels, title=condition_col, fontsize=14, title_fontsize=16)

        plt.tight_layout()
        plt.show()

# 以下の「hoge.csv」の場所にて、CSVファイルの名前を書き換えてください。
plot_abc_bc_patient_data('hoge.csv')
